# **Image segmentation**
The goal of image segmentation is to turns image in a more symbolic representation.

Denoted as $P(x,y)$ a vector-valued function encoding a set of image properties (intensity, colour, texture, etc...), the goal of segmentation is to partition the image into disjoint homogeneous regions according to P.

A good segmentation should preserve spatial proximity and provide relatively large regions, featuring a few holes and well-localized smooth boundaries.

In many computer vision tasks, segmentation splits the image into semantically meaningful parts, on which further analysis can be done.

In many pratical application of *machine vision*, segmentation relies on just a single image property, such as:
* The intensity $P(x,y) = I(x,y)$;
* The colour $P(x,y) = {\begin{bmatrix} I_r(x,y) I_g(x,y) I_b(x,y) \end{bmatrix}}^T$.

Some images are inherently-binary, i.e. qr codes or barcodes.

## Image binarization
In a variety of applications the objects of interest (*foreground*) are neatly darker or brighter than the irrelevant area of the scene (*background*).
The segmentation of the image pixels into these two disjoint regions is called *binarization*.

**Backlighting**: the object is placed between a camera and a light source, in order to cast a very dark shadow representig the object's shape.

The foreground region could be further split into sub-regions, corresponding to individual objects (for apply an *image labeling*).

## Binarization by intensity thresholding
Inherently-binary images exhibit a clearly bimodal gray-level histogram: two well separated peaks representing foreground and background:
<center> <img src=https://i.ibb.co/LJfRZFd/photo-2021-01-22-09-39-52.jpg width="800px" /> </center>

In this case, binarization can be achieved straightforwardly by a *thresholding*.
There are some cases where, i.e. due to bad illumination, thresholding fails to provide the correct segmentation:
<center> <img src=https://i.ibb.co/RBT8RvQ/photo-2021-01-22-09-46-24.jpg width="700px" /> </center>

There are some other cases where the overlap between the due modes it's due to noise:
<center> <img src=https://i.ibb.co/QJQqjxs/photo-2021-01-22-09-50-46.jpg width="500px" /> </center>

In these cases a previous image smoothing may help (if the lighting is stable).

## Automatic threshold selection
When stability over the time of the lighting conditions can't be guaranteed, a more robust solutions (and more expensive) are required.

1. Mean intensity thresholding $T = \mu$, this method works as long as pixels are equally distribuited between the two classes (otherwise a percentile may be used):
<center> <img src=https://i.ibb.co/FmzrFqC/photo-2021-01-22-09-57-40.jpg width="700px" /> </center>

1. Peaks method $T = \text{argmin} \bigl\{ h(i) | i \in [i_1,i_2] \bigr\}$, select the point where the histogram is $0$ between the two peaks (hypothesis of a bimodal histogram).
A previous smoothing is done to avoid the search to be stuck in spurious local maxima:
<center> <img src=https://i.ibb.co/WsFfnCw/photo-2021-01-22-10-02-47.jpg width="400px" /> </center>

* 3-levels neighbourhood `((h[i] > h[i-1]) && (h[i] > h[i+1]))`;
* 5-levels neighbourhood `((h[i] > h[i-1]) && (h[i] > h[i+1]) && (h[i-1] > h[i-2]) && (h[i+1] > h[i+2]))`.


### Otsu's algorithm
The key intuition is to segment the image into two maximally homogeneous regions.
The optimal threshold is chosen to minimize, across the gray-level range, the **within-group variance** of the resulting regions (it's an indicator that measures how spread are the region after the binarization).

*Note*: a class is said to be homogeneous when the variance of the samples in the class is small.

Let be:
* $i = 1 \dots L$ the gray-levels of the image;
* $N$ the total number of pixels of the image;
* $h(i)$ the $i$-th entry of the image histogram;
* $p(i) = \frac{h(i)}{N}$ the probability of the gray-level $i$. Remember that the normalization condition $\sum_{i=1}^L p(i) = 1$ holds.

The mean $\mu$ and the variance $\sigma^2$ of the *pmf* associated with the image gray-levels can be expressed as:
$$\mu = \sum_{i = 1}^L i p(i) \quad \quad \sigma^2 = \sum_{i = 1}^L (i - \mu)^2 p(i)$$

If we split with a certain threshold $t$ we will have:
$$\mu_1 = \sum_{i = 1}^t i \frac{p(i)}{q_1(t)} \quad \quad \sigma^2_1 = \sum_{i = 1}^L (i - \mu_1)^2 \frac{p(i)}{q_1(t)}$$
$$\mu_2 = \sum_{i = t + 1}^L i \frac{p(i)}{q_2(t)} \quad \quad \sigma^2_2 = \sum_{i = t+1}^L (i - \mu_2)^2 \frac{p(i)}{q_2(t)}$$

where $q_1(t)$ and $q_2(t)$ are the percentages of pixels in the same area:
$$ q_1(t) = \sum_{i=1}^t p(i) \quad \quad q_2(t) = \sum_{i=t+1}^L p(i)$$

The above relation can be found as follow:
<center> <img src=https://i.ibb.co/DQw33M5/photo-2021-01-22-10-26-38.jpg width="400px" /> </center>

The **within-group variance** of the two regions is defined as **the weighted sum of their variances**:
$$\sigma_W^2 = q_1(t) \sigma_1^2(t) + q_2(t) \sigma_2^2(t)$$

The minimization (1d space search) would require to compute $\mu_1$, $\mu_2$, $\sigma_1^2$, $\sigma_2^2$ and $q_1$ (since $q_2 = 1 - q_1$) for each gray-level values.

A more efficient approach can be deployed (especially if the search space is important).

Let's consider:
$$\sigma^2 = \sum_{i = 1}^L (i - \mu)^2 p(i)$$
split the sum in two terms:
$$\sigma^2 = \sum_{i = 1}^t (i - \mu)^2 p(i) + \sum_{i = t + 1}^L (i - \mu)^2 p(i)$$
subtract and add $\mu_1(t)$ and $\mu_2(t)$ to each sum:
$$\sigma^2 = \sum_{i = 1}^t (i - \mu_1(t) + \mu_1(t) - \mu)^2 p(i) + \sum_{i = t + 1}^L (i - \mu_2(t) + \mu_2(t) - \mu)^2 p(i)$$
and calculate the binomial square:
$$\sigma^2 = \sum_{i = 1}^t \Bigl[(i - \mu_1(t))^2 + (\mu_1(t) - \mu)^2 - 2(i - \mu_1(t))(\mu_1(t) - \mu)\Bigr] p(i) + \sum_{i = t + 1}^L \Bigl[(i - \mu_2(t))^2 + (\mu_2(t) - \mu)^2 - 2(i - \mu_2(t))(\mu_2(t) - \mu) \Bigr] p(i)$$

It can be shown that the double product is null, hence:
$$\sigma^2 = \sum_{i = 1}^t (i - \mu_1(t))^2 p(i) + \sum_{i = t + 1}^L (i - \mu_2(t))^2 p(i) + \Bigl[(\mu_1(t) - \mu)^2 q_1(t) + (\mu_2(t) - \mu)^2 q_2(t) \Bigr]$$

$$\sigma^2 = \Bigl[ q_1(t) \sigma_1^2 + q_2(t) \sigma_2^2 \Bigr] + \Bigl[(\mu_1(t) - \mu)^2 q_1(t) + (\mu_2(t) - \mu)^2 q_2(t) \Bigr]$$

$$\sigma^2 = \sigma_W^2 + \sigma_B^2$$

Where $\sigma_B^2$ is the **between-group variance**, an indicator that measure how well classes are separated.

Since $\sigma^2$ is independent of $t$ we could maximize $\sigma_B^2$ rather than minimize $\sigma_W^2$.
This is more efficient since variances doesn't need to be calculated.

Further computational savings can be achieved with an incremental calculation:
$$\mu = q_1(t) \mu_1(t) + q_2(t) \mu_2(t)$$

$$\mu = q_1(t) \mu_1(t) + [1 - q_1(t)] \mu_2(t)$$
so we'll have:
$$\sigma_B^2 = q_1(t) (1 - q_1(t)) (\mu_1(t) - \mu_2(t))^2$$

The calculations can be done in an incremental way:
$$q_1(t+1) = q_1(t) + p(t+1)$$

$$\mu_1(t+1) =\frac{q_1(t)\mu_1(t) + (t+1)p(t+1)}{q_1(t+1)}$$

$$\mu_2(t+1) =\frac{\mu - q_1(t+1) \mu_1(t+1)}{1 - q_1(t+1)}$$

*Example*
<center> <img src=https://i.ibb.co/23nS9Wn/thres.jpg width="700px" /> </center>

### Adaptive thresholding
Any global thresholding method relies on the assumption of uniform lighting across the scene.
If this assumption is violated, a spatially varying threshold should be used.

Usually, adaptive methods compute a specific threshold at each image pixel, based on the intensity of a small neighbourhood (too small neighbourhood might lack of either background or foreground pixels, which would imply segmentation errors).

For sake of efficiency, simple operators such as mean or median are used to compute the threshold at each pixel.

<center> <img src=https://i.ibb.co/rdHvGHj/photo-2021-01-22-11-23-35.jpg width="700px" /> </center>

In text images, the problem of the lack of both kind of pixels in the neighbourhood can be solved exploiting the peculiar fact that most of pixels are background pixels.

A simple trick to improve drastically segmentation is to subtract a tuned constant: 
$$T(x,y) = \mu (x,y) - C$$

In this manner we push above the threshold most wrongly classified background pixels.


<center> <img src=https://i.ibb.co/kSZ6B9b/photo-2021-01-22-11-31-01.jpg width="700px" /> </center>

## Colour-based segmentation
Tipically it's not used in machine vision since it requires a more expensive hardware and an heavier computation.
It's used when the objects have a known colour, quite different from the background, or viceversa (for example in food conveyors is used a blue belt, since food is rarely blue).

Let's consider a pixel's colour as:
$$I(p) = \begin{bmatrix} I_r(p) \\ I_g(p) \\ I_b(p) \end{bmatrix}$$
where $p$ is a pixel coordinate $p \to (u,v)$.

The thresholding can be achieved by computing and thresholding the distance between each pixel's colour and the reference background (or foreground) colour $\mu$:
$$\forall p \in I \colon \quad \begin{cases} d\bigl( I(p), \mu \bigr) \leq T \quad \to \quad O(p) = B \\ d\bigl( I(p), \mu \bigr) > T \quad \to \quad O(p) = F \end{cases}$$

The distance can be any kind of metric, for example the Euclidean distance is:
$$d\bigl( I(p), \mu \bigr) = {\Bigl({\bigl( I_r(p) - \mu_r \bigr)}^2 + {\bigl( I_g(p) - \mu_g \bigr)}^2 + {\bigl( I_b(p) - \mu_b \bigr)}^2 \Bigr)}^{\frac{1}{2}}$$
that can be rewritten in a vectorized form (more suitable for hardware):
$$d = ||I(p) - \mu|| = {\bigl[ (I(p) - \mu)^T (I(p) - \mu) \bigr]}^{\frac{1}{2}}$$

### Colour estimation
It's necessary to know the reference colour $\mu$, that can be learned from one or more training images.

Modelling the colour of a backgroud (ore foreground) pixel can be modelled as a multivariate random variable, hence the reference colour can be taken to be the mean over all the available training samples:
$$\mu = \begin{bmatrix} \mu_r \\ \mu_g \\ \mu_b \end{bmatrix} = \frac{1}{N} \sum_{k = 1}^N I(p_k)$$

*Note*: for a better estimation we could use a further order of expectation, such as the covariance.

The segmentation will consists in classifyng as background (or foreground) all pixels lying within a 3d sphere of the RGB colour space, centered at $\mu$ and having radius $T$ (isotropic decision surface, there are other better approaches).

<center> <img src=https://i.ibb.co/Sc9JbWZ/photo-2021-01-22-11-57-43.jpg width="300px" /> </center>

### Mahalanobis distance
A way richer probabilistic characterization of the colour distribution can be obtained estimating not only the mean, but also the covariance matrix:
$$\Sigma = \begin{pmatrix} \sigma_{rr}^2 & \sigma_{rg}^2 & \sigma_{rb}^2 \\ \sigma_{gr}^2 & \sigma_{gg}^2 & \sigma_{gb}^2 \\ \sigma_{br}^2 & \sigma_{bg}^2 & \sigma_{bb}^2 \end{pmatrix}$$

*Note*: in the diagonal there are crossvariances.

We can rewrite the general term as:
$$\sigma_{ij}^2 = \frac{1}{N} \sum_{k = 1}^N \bigl(I_i (p_k) - \mu_i \bigr) \bigl(I_j(p_k) - \mu_j \bigr) \quad \quad i,j \in \{r,g,b\}$$

In this way we can capture the correlations between colour channels.

Recalling that the Euclidean distance can be written as:
$$d\bigl(I(p), \mu \bigr) = {\bigl[ (I(p) - \mu)^T (I(p) - \mu) \bigr]}^{\frac{1}{2}}$$
then, the Mahalanobis distance can be written simply as:
$$d_M \bigl(I(p), \mu \bigr) = {\bigl[ (I(p) - \mu)^T \Sigma^{-1} (I(p) - \mu) \bigr]}^{\frac{1}{2}}$$

The decision surface is ellipsoidal rather than spherical.



#### Understanding Mahalanobis distance
To get the difference between the two metrics, let's consider without loss of generalization the case of a diagonal covariance matrix (we can always diagonalize it):
$$\Sigma = \begin{pmatrix} \sigma_{rr}^2 & 0 & 0 \\ 0 & \sigma_{gg}^2 & 0 \\ 0 & 0 & \sigma_{bb}^2 \end{pmatrix} \quad \to \quad \Sigma^{-1} = \begin{pmatrix} \frac{1}{\sigma_{rr}^2} & 0 & 0 \\ 0 & \frac{1}{\sigma_{gg}^2} & 0 \\ 0 & 0 & \frac{1}{\sigma_{bb}^2} \end{pmatrix}$$

Hence:
<center> <img src=https://i.ibb.co/2Pzn5fh/photo-2021-01-22-12-14-35.jpg width="800px" /> </center>

The Mahalanobis distance weighs unequally the differences along the components of the random vectors (it weighs according to inverse proportionality to the learned variances).
The more spread has been learned to a component, the lesse the difference along that component will contribuite to the overall distance.

Now the thresholding is determined by:
$$d_M \bigl(I(p), \mu \bigr)^2 = \frac{{\bigl( I_r(p) - \mu_r \bigr)}^2}{\sigma_{rr}^2} + \frac{{\bigl( I_g(p) - \mu_g \bigr)}^2}{\sigma_{gg}^2} + \frac{{\bigl( I_b(p) - \mu_b \bigr)}^2}{\sigma_{bb}^2} \leq T^2$$

So the decision surface is an ellipsoid centered at:
$$\mu = \begin{bmatrix} \mu_r \\ \mu_g \\ \mu_b \end{bmatrix} $$
with axes aligned to coordinate axes, and semi-axes lenght given by:
$$L = \begin{bmatrix} L_r \\ L_b \\ L_g \end{bmatrix} = T \begin{bmatrix} \sigma_{rr} \\ \sigma_{bb} \\ \sigma_{gg} \end{bmatrix} $$

<center> <img src=https://i.ibb.co/dDkWkZM/photo-2021-01-22-12-26-50.jpg width="300px" /> </center>

*Example*
<center> <img src=https://i.ibb.co/d041jGh/photo-2021-01-22-12-28-51.jpg width="800px" /> </center>